In [22]:
import geopandas as gpd
import numpy as np
import datetime
import os
import rasterio

In [2]:
import sys
sys.path.append('..')

In [3]:
import config
import create_s2l1c_datacube
import rsutils.s2_grid_utils

/Users/nikhilsrajan/NASA-Harvest/project/fetch_satdata/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
roi_filepath = '../../ethiopia/data/Ethiopia_Bounding_Boxs/Ethiopia_2021_East_Bounding_Box.shp'

In [5]:
test_grid_id = '17b43dc'
mosaic_days_interval = 20

In [6]:
# shapes_gdf = gpd.read_file(roi_filepath).to_crs('epsg:4326')
shapes_gdf = gpd.GeoDataFrame(data={
    'id': [test_grid_id],
    'geometry': [rsutils.s2_grid_utils.get_grid_geometry_from_id(test_grid_id)],
}, crs='epsg:4326')
shapes_gdf

,id,geometry
0,17b43dc,"POLYGON ((38.98559 7.34009, 39.03147 7.33538, ..."


In [7]:
startdate = datetime.datetime(2021, 3, 1)
enddate = datetime.datetime(2021, 4, 1)

In [9]:
export_folderpath = f'../data/test_s2l2a_datacube/{test_grid_id}/{mosaic_days_interval}'

In [16]:
create_s2l1c_datacube.create_s2l2a_datacube(
    shapes_gdf = shapes_gdf,
    export_folderpath = export_folderpath,
    satellite_catalog_filepath = '../data/satellite/Sentinel-2/catalog.geojson',
    startdate = startdate,
    enddate = enddate,
    njobs = 8,
    mosaic_days = mosaic_days_interval,
    bands = [
        'B01', 'B02', 'B03', 'B04',
        'B05', 'B06', 'B07', 'B08',
        'B8A', 'B09', 'B11',
        'B12', 'SCL',
    ]
)

Cropping tiles and reprojecting to common CRS:


100%|██████████| 156/156 [00:06<00:00, 25.17it/s]


Resampling cropped images to resolution of B08 band:


100%|██████████| 12/12 [00:03<00:00,  3.17it/s]


Resampling cropped images to merged shape:


100%|██████████| 156/156 [00:04<00:00, 37.77it/s]


Extracting mean_sun_angle:


100%|██████████| 12/12 [00:00<00:00, 82.58it/s]

Running apply_cloud_mask_scl -- kwargs={'mask_classes': [0, 1, 3, 7, 8, 9, 10]}


Running drop_bands -- kwargs={'bands_to_drop': ['SCL']}
Running median_mosaic -- kwargs={'startdate': datetime.datetime(2021, 3, 1, 0, 0), 'enddate': datetime.datetime(2021, 4, 1, 0, 0), 'mosaic_days': 20}


In [17]:
datacube = np.load(os.path.join(export_folderpath, 'datacube.npy'))
metadata = np.load(os.path.join(export_folderpath, 'metadata.pickle.npy'), allow_pickle=True)[()]

In [18]:
datacube.shape

(2, 478, 508, 12)

In [19]:
metadata

{'geotiff_metadata': {'driver': 'GTiff',
  'dtype': 'uint16',
  'nodata': 0,
  'width': 508,
  'height': 478,
  'count': 1,
  'crs': CRS.from_epsg(32637),
  'transform': Affine(10.0, 0.0, 498400.0,
         0.0, -10.0, 815600.0),
  'compress': 'lzw'},
 'timestamps': [Timestamp('2021-03-04 07:57:49.172000+0000', tz='UTC'),
  Timestamp('2021-03-24 07:57:49.086000+0000', tz='UTC')],
 'ids': ['S2B_MSIL2A_20210304T073809_N0500_R092_T37NEJ_20230529T142251.SAFE',
  'S2B_MSIL2A_20210304T073809_N0500_R092_T37NDJ_20230529T142251.SAFE',
  'S2A_MSIL2A_20210309T073741_N0500_R092_T37NEJ_20230523T144831.SAFE',
  'S2A_MSIL2A_20210309T073741_N0500_R092_T37NDJ_20230523T144831.SAFE',
  'S2B_MSIL2A_20210314T073659_N0500_R092_T37NEJ_20230529T050002.SAFE',
  'S2B_MSIL2A_20210314T073659_N0500_R092_T37NDJ_20230529T050002.SAFE',
  'S2A_MSIL2A_20210319T073621_N0500_R092_T37NEJ_20230604T084114.SAFE',
  'S2A_MSIL2A_20210319T073621_N0500_R092_T37NDJ_20230604T084114.SAFE',
  'S2B_MSIL2A_20210324T073609_N0500_R092_T

In [20]:
band_indices = dict(zip(metadata['bands'], range(len(metadata['bands']))))

In [21]:
band_indices

{'B01': 0,
 'B02': 1,
 'B03': 2,
 'B04': 3,
 'B05': 4,
 'B06': 5,
 'B07': 6,
 'B08': 7,
 'B8A': 8,
 'B09': 9,
 'B11': 10,
 'B12': 11}

In [23]:
out_meta = metadata['geotiff_metadata']
out_meta

{'driver': 'GTiff',
 'dtype': 'uint16',
 'nodata': 0,
 'width': 508,
 'height': 478,
 'count': 1,
 'crs': CRS.from_epsg(32637),
 'transform': Affine(10.0, 0.0, 498400.0,
        0.0, -10.0, 815600.0),
 'compress': 'lzw'}

In [25]:
with rasterio.open(os.path.join(export_folderpath, 'B08_20_day_mosaic.tif'), 'w', **out_meta) as dst:
    dst.write(np.expand_dims(datacube[0,:,:,band_indices['B08']], axis=0))